# 데이터 불러오기부터

### 가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

#### [참고] AIDU에서 필요한 추가 라이브러리 Import : 아래 코드는 AIDU에서만 실행 가능합니다.

In [ ]:
# from aicentro.session import Session
# from aicentro.framework.tensorflow import tensorflow as AiduFrm

### 나. 데이터 불러오기

In [ ]:
df_evaluation = pd.read_csv("onenavi_evaluation_new.csv",sep="|")
df_evaluation_feature = pd.read_csv("onenavi_eval_feature.csv",sep="|")

In [ ]:
df_evaluation_feature

#### [참고] AIDU에서는 파일을 불러올 때 아래 예시를 참고 : 아래 코드는 AIDU에서만 실행 가능합니다.

In [ ]:
# # 데이터 셋 로딩
# aicentro_session = Session(verify = False)
# aidu_framework = AiduFrm(session = aicentro_session)
# df_evaluation = pd.read_csv(aidu_framework.config.data_dir+"onenavi_evaluation_new.csv",sep="|")
# df_evaluation_feature = pd.read_csv(aidu_framework.config.data_dir+"onenavi_eval_feature.csv",sep="|")
# df_evaluation_feature # 데이터 프레임 확인하기

# 1. 저장된 모델 불러오기

### 가. 라이브러리 Import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as grb
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

import pickle
import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping

### 나. 평가데이터 준비하기

In [ ]:
len(df_evaluation_feature)

In [ ]:
df_evaluation_feature

### 다. 모델 불러오기

In [ ]:
# 모델을 담을 빈 리스트 생성
model_rslt = []

# 앞서 저장한 머신러닝 모델 불러오기 및 저장
for i in range(4):
    model_rslt.append(joblib.load("{}_model.pkl".format(i)))
# 앞서 저장한 딥러닝 모델 불러오기 및 저장
model_rslt.append(keras.models.load_model("DeeplearningModel.h5"))

In [ ]:
# 저장된 모델 확인
model_rslt

### 라. 모델 결과 표 만들기

In [ ]:
e1_list = ['ETA1', 'ETA2', 'ETA3', 'ETA4', 'ETA5']
e2_list = ['ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5']

for e1, e2, model in zip(e1_list, e2_list, model_rslt):
    df_evaluation[e1] = model.predict(df_evaluation_feature)
    df_evaluation.loc[(df_evaluation[e1] < 0), e1] = 0
    etaa = (1-(abs(df_evaluation['ET']-df_evaluation[e1])/df_evaluation['ET']))*100.0
    df_evaluation[e2] = etaa
    df_evaluation.loc[(df_evaluation[e2] < 0), e2] = 0

# mean, min, max, std
etaa = ['ETAA', 'ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5']
alg = ['DATA', 'ML-LG', 'ML-RFR', 'ML-GBR', 'XBR', 'Deep']

print('+-------------------------------------------------------+')
print('|   ALG    | Mean(%) |    STD    |  MIN(%)  |  MAX(%)   |')
print('+----------+---------+-----------+----------+-----------+')
for i, e in zip(range(len(alg)), etaa):
    eMean = df_evaluation[e].mean()
    eStd = df_evaluation[e].std()
    eMin = df_evaluation[e].min()
    eMax = df_evaluation[e].max()
    print('|  {:6s}  |   {:3.1f}  |   {:05.1f}   |   {:4.1f}   |  {:7.1f}  | '.format(alg[i], eMean, eStd, eMin, eMax))
print('+----------+---------+-----------+----------+-----------+\n\n')

### 좀 더 상세한 분석을 위해 거리 구간별로 결과를 확인해 보겠습니다.

In [ ]:
etaa = ['ETAA', 'ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5']
alg = ['DATA', 'ML-LG', 'ML-RFR', 'ML-GBR', 'XBR', 'Deep']

for length in range(4):
    if length == 0:
        print(' 1,000 <= A_DISTANCE < 5,000m')
    elif length == 1:
        print(' 5,000 <= A_DISTANCE < 10,000m')
    elif length == 2:
        print(' 10,000 <= A_DISTANCE < 15,000m')
    else:
         print('All A_DISTANCE')
    print('+-------------------------------------------------------+')
    print('|   ALG    | Mean(%) |    STD    |  MIN(%)  |  MAX(%)   |')
    print('+----------+---------+-----------+----------+-----------+')
    for i, e in zip(range(len(alg)), etaa):
        if length == 0:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].max()
        elif length == 1:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].max()
        elif length == 2:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].max()
        else:
            eMean = df_evaluation[e].mean()
            eStd = df_evaluation[e].std()
            eMin = df_evaluation[e].min()
            eMax = df_evaluation[e].max()
        print('|  {:6s}  |   {:3.1f}  |   {:05.1f}   |   {:4.1f}   |  {:7.1f}  |'.format(alg[i], eMean, eStd, eMin, eMax))
    print('+----------+---------+-----------+----------+-----------+\n\n')

### 100개의 샘플로 결과를 시각화를 시도하겠습니다.

In [ ]:
plt.figure(figsize=(100, 10))
idx = np.arange(0, 100)
plt.plot(idx, df_evaluation.ET[0:100], 'b', marker='s')
plt.plot(idx, df_evaluation.ETA1[0:100], 'g', marker='s')
plt.plot(idx, df_evaluation.ETA2[0:100], 'r', marker='+')
plt.plot(idx, df_evaluation.ETA3[0:100], 'y', marker='*')
plt.plot(idx, df_evaluation.ETA4[0:100], 'b', marker='o')
plt.plot(idx, df_evaluation.ETA5[0:100], 'b', marker='^')
plt.plot()
plt.xlabel('Index')
plt.ylabel('ET(sec)')
plt.legend(['Real', 'ML-LG', 'ML-RFR', 'ML-GBR', 'XBR', 'Deep'])
plt.show()